In [ ]:
!pip install -q py_vncorenlp

import py_vncorenlp
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt 
import re
import pandas as pd
import warnings
import os
import random

warnings.filterwarnings(
    "ignore",
    message="Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.",
    category=UserWarning
)

def set_global_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_global_seed(42)

In [ ]:
label2id = {
    "gambling": 0,
    "movies": 1,
    "ecommerce": 2,
    "government": 3,
    "education": 4,
    "technology": 5,
    "tourism": 6,
    "health": 7,
    "finance": 8,
    "media": 9,
    "nonprofit": 10,
    "realestate": 11,
    "services": 12,
    "industries": 13,
    "agriculture": 14
}


id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)

model_name = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id)
model.to(device)
print(f"Model '{model_name}' and tokenizer loaded successfully with {num_labels} labels.")

In [ ]:
# Ensure the directory exists before downloading
os.makedirs('/kaggle/working/vncorenlp', exist_ok=True)

# Download VnCoreNLP model
py_vncorenlp.download_model(save_dir='/kaggle/working/vncorenlp')

# Load the segmenter
rdrsegmenter = py_vncorenlp.VnCoreNLP(
    annotators=["wseg"], 
    save_dir='/kaggle/working/vncorenlp'
)

# Test word segmentation
text = "Ông Nguyễn Khắc Chúc đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."
output = rdrsegmenter.word_segment(text)

print(output)

In [ ]:
dataset = load_dataset('csv', data_files='insert-path')
dataset = dataset['train']
print('dataset loaded!')

def encode_labels(examples):
    examples['label'] = [label2id[label] for label in examples['label']]
    return examples

dataset = dataset.map(encode_labels, batched=True)
print("Labels encoded to numerical IDs.")

token_lengths = []

def preprocess_function(examples):
    # Apply Vietnamese word segmentation
    segmented_texts = [' '.join(rdrsegmenter.word_segment(text)) for text in examples['text']]
    
    # Tokenize segmented text
    tokenized_inputs = tokenizer(
        segmented_texts,
        truncation=True,
        padding='max_length',
        max_length=64
    )
    
    # Count non-padding tokens
    for input_ids in tokenized_inputs['input_ids']:
        length = len([token_id for token_id in input_ids if token_id != tokenizer.pad_token_id])
        token_lengths.append(length)
    
    return tokenized_inputs
    
tokenized_dataset = dataset.map(preprocess_function, batched=True, load_from_cache_file=False)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")    # Hugging Face requires a specific column named 'labels'
tokenized_dataset.set_format("torch")

features = tokenized_dataset
labels = [label.item() for label in tokenized_dataset['labels']]

train_indices, test_indices = train_test_split(
    range(len(labels)),
    test_size=0.1,
    random_state=42,
    stratify=labels
)

train_dataset = features.select(train_indices)
eval_dataset = features.select(test_indices)


max_length = max(token_lengths)
min_length = min(token_lengths)
avg_length = sum(token_lengths) / len(token_lengths)

print(f"Token length stats before padding:")
print(f"  🔹 Max length: {max_length}")
print(f"  🔹 Min length: {min_length}")
print(f"  🔹 Avg length: {avg_length:.2f} tokens")
print(f"Dataset tokenized and split into {len(train_dataset)} training examples and {len(eval_dataset)} evaluation examples.")

labels = [label.item() for label in train_dataset['labels']]
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Emphasize 
# scale_factor = 1.0
# class_weights[0] *= scale_factor

In [ ]:
# Plot histogram of token length distribution (after truncation)
plt.figure(figsize=(8, 5))
plt.hist(token_lengths, bins=50, color='skyblue', edgecolor='black')
plt.title("Token Length Distribution (Before Padding)")
plt.xlabel("Token Length")
plt.ylabel("Number of Samples")
plt.axvline(x=64, color='red', linestyle='--', label='max_length=64')
plt.legend()
plt.show()

In [ ]:
# # After finish evaluation, re-train on the whole dataset to maximise learning (intentional data leakage)
# train_dataset = tokenized_dataset
# print(f"Training on the entire dataset with {len(train_dataset)} examples.")

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, class_weights=None, gamma=2.0, reduction='mean', label_smoothing=0.2):
        super(FocalLoss, self).__init__()
        self.class_weights = class_weights
        self.gamma = gamma
        self.reduction = reduction
        self.label_smoothing = label_smoothing

    def forward(self, logits, targets):
        num_classes = logits.size(1)
        smoothed_labels = F.one_hot(targets, num_classes).float()
        smoothed_labels = smoothed_labels * (1 - self.label_smoothing) + self.label_smoothing / num_classes

        log_probs = F.log_softmax(logits, dim=1)
        ce_loss = -(smoothed_labels * log_probs).sum(dim=1)

        if self.class_weights is not None:
            weights = self.class_weights[targets]
            ce_loss = ce_loss * weights

        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        return focal_loss.mean() if self.reduction == 'mean' else focal_loss.sum()

class CustomTrainer(Trainer):
    def __init__(self, *args, focal_loss=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss = focal_loss

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.focal_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
focal_loss = FocalLoss(class_weights=class_weights, gamma=2.0, label_smoothing=0.2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="no",
    load_best_model_at_end=False,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=42,
    report_to="none"
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average='weighted'),
        "recall": recall_score(labels, preds, average='weighted'),
        "f1": f1_score(labels, preds, average='weighted')
    }

In [ ]:
# # Reload model if retraining
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id)
# model.to(device)

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    focal_loss=focal_loss,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.001)]
)

In [ ]:
print("Starting training...")
trainer.train()
print("Training complete!")

In [ ]:
model_save_path = "./fine_tuned_phobert"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Fine-tuned model saved to {model_save_path}")

In [ ]:
# # Empty the working directory if retraining
# import os
# import shutil

# working_dir = "/kaggle/working/"
# print(f"Attempting to clear the directory: {working_dir}")

# if os.path.exists(working_dir):
#     for item in os.listdir(working_dir):
#         item_path = os.path.join(working_dir, item)
#         try:
#             if os.path.isfile(item_path) or os.path.islink(item_path):
#                 os.unlink(item_path)
#             elif os.path.isdir(item_path):
#                 shutil.rmtree(item_path)
#         except Exception as e:
#             print(f"Error removing {item_path}: {e}")
#     print(f"Contents of {working_dir} cleared.")
# else:
#     print(f"Directory {working_dir} does not exist.")

# if not os.listdir(working_dir):
#     print(f"Directory {working_dir} is now empty.")
# else:
#     print(f"Directory {working_dir} still contains items: {os.listdir(working_dir)}")

In [ ]:
# Evaluate the test set
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
# # Plot the graphs
# log_history = trainer.state.log_history

# epochs = []
# train_losses = []
# eval_losses = []
# eval_accuracies = []
# eval_f1_scores = []
# eval_precisions = []
# eval_recalls = []

# for log in log_history:
#     if 'loss' in log and 'learning_rate' in log and 'epoch' in log:
#         train_losses.append(log['loss'])
#         epochs.append(log['epoch'])
#     elif 'eval_loss' in log:
#         eval_losses.append(log['eval_loss'])
#         eval_accuracies.append(log['eval_accuracy'])
#         eval_f1_scores.append(log['eval_f1'])
#         eval_precisions.append(log['eval_precision'])
#         eval_recalls.append(log['eval_recall'])

# eval_epochs = [log['epoch'] for log in log_history if 'eval_loss' in log]

# # Plotting Loss
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(epochs[:len(train_losses)], train_losses, label='Training Loss')
# plt.plot(eval_epochs, eval_losses, label='Validation Loss')
# plt.title('Loss over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(True)

# # Plotting Accuracy and F1-score
# plt.subplot(1, 2, 2)
# plt.plot(eval_epochs, eval_accuracies, label='Validation Accuracy')
# plt.plot(eval_epochs, eval_f1_scores, label='Validation F1-score (Weighted)')
# plt.plot(eval_epochs, eval_precisions, label='Validation Precision (Weighted)')
# plt.plot(eval_epochs, eval_recalls, label='Validation Recall (Weighted)')
# plt.title('Metrics over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Score')
# plt.legend()
# plt.grid(True)

# plt.tight_layout() 
# plt.show()

In [ ]:
# Save misclassified samples to CSV

import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm

# Get predictions on eval_dataset
predictions_output = trainer.predict(eval_dataset)

# Convert logits to predicted class and confidence
logits = predictions_output.predictions
probs = torch.softmax(torch.tensor(logits), dim=1)
predicted_labels = torch.argmax(probs, axis=1)
confidences = torch.max(probs, axis=1).values

# Get ground truth labels
true_labels = predictions_output.label_ids

# Get original text from dataset
original_texts = [tokenizer.decode(inputs['input_ids'], skip_special_tokens=True) for inputs in eval_dataset]

# Map IDs to label names
true_label_names = [id2label[label] for label in true_labels]
predicted_label_names = [id2label[label.item()] for label in predicted_labels]

# Filter misclassified samples
misclassified = []
for text, true, pred, conf in zip(original_texts, true_label_names, predicted_label_names, confidences):
    if true != pred:
        misclassified.append({
            "original_text": text,
            "true_label": true,
            "predicted_label": pred,
            "confidence": round(conf.item(), 4)
        })

# Save to CSV
df_misclassified = pd.DataFrame(misclassified)
df_misclassified.to_csv("misclassified_samples.csv", index=False, encoding="utf-8-sig")
print(f"Saved {len(df_misclassified)} misclassified samples to misclassified_samples.csv")

# Inference

In [ ]:
def clean_text(text):
    if pd.isna(text):
        return text
    
    #text = text.lower()

    # === Preserve domain dots, decimal dots, and URL hyphens ===
    text = re.sub(r'(\w)\.(?=\w)', r'\1<DOMAIN>', text)      # domain dots
    text = re.sub(r'(\d)\.(?=\d)', r'\1<DECIMAL>', text)     # decimal dots
    text = re.sub(r'(\w)-(?=\w)', r'\1<HYPHEN>', text)       # hyphen inside words/domains

    # === Remove remaining dots and hyphens ===
    text = text.replace('.', '')
    text = text.replace('-', '')

    # === Replace one or more underscores with a single space ===
    text = re.sub(r'_+', ' ', text)

    # === Restore preserved characters ===
    text = text.replace('<DOMAIN>', '.')
    text = text.replace('<DECIMAL>', '.')
    text = text.replace('<HYPHEN>', '-')

    # === Handle commas ===
    text = re.sub(r'(?<=[a-z0-9]),(?=[a-z])', ' ', text)  # digit/letter → letter
    text = re.sub(r'(?<=[a-z]),(?=[0-9])', ' ', text)     # letter → digit
    text = re.sub(r',(?=\D)|(?<=\D),', '', text)          # remove other commas

    # === Remove unwanted punctuation (keep quotes, %, /) ===
    text = re.sub(r'[^\w\s\.,/%"]', '', text)

    # === Normalize spaces ===
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
model.eval()

# Testing manually
def predict_text_class(text_input: str):
    # Clean the text first
    # text = clean_text(text_input)

    # Apply Vietnamese word segmentation (same as training)
    segmented_text = ' '.join(rdrsegmenter.word_segment(text))

    # Tokenize segmented text
    inputs = tokenizer(
        segmented_text,
        return_tensors="pt",
        truncation=True,
        padding='max_length',
        max_length=64
    )

    # Move tensors to the correct device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    predicted_label = id2label[predicted_class_id]

    return predicted_label


example_texts = [
    "Hệ thống nha khoa Tâm Đức Smile dẫn đầu về các dịch vụ cấy ghép Implant, răng sứ thẩm mỹ, niềng răng uy tín hàng đầu, ưu đãi lên đến 60%.",
    "Nhà Xe Mỹ Duyên - Vận chuyển hành khách và hàng hóa chuyên tuyến Hồ Chí Minh đi Sóc Trăng và ngược lại Nhận vé Nhận mã vé, xác nhận và lên xe",
    "Công ty tư vấn du học Mỹ cập nhật mới nhất về điều kiện, chi phí, hồ sơ xin visa du học mỹ, cơ hội định cư. Công ty du học Á - Âu 24 năm kinh nghiệm, đỉnh cao uy tín và chất lượng.",
    "máy rung cầm tay tặng bạn gái",
    "thd cybersecurity",
    "Vay Cầm Cố, Giải Ngân 15p Tổng hợp các bài viết từ cơ quan báo chí truyền thông uy tín, phản ánh hoạt động, thành tựu nổi bật và chiến lược phát triển của F88.",
    "Đường dây đánh bạc, nỗ hủ, cá độ bóng đá rửa tiền lên đến 1.000 tỷ đồng bị triệt phá",
    "Samsung Galaxy Z Fold7 | Foldable meets Ultra Sleek | Samsung Australia",
    "đường vào tim em ôi băng giá",
    "Nghị định số 46/2017/NĐ-CP ngày 21/4/2017 của Chính phủ quy định về hoạt động đầu tư giáo dục trong các chương trình đào tạo mầm non, phổ thông, đại học; ...",
    "5 ngày trước — tachi,Nền tảng xổ số trực tuyến với tỷ lệ hoa hồng cho đại lý cao nhất. Hợp tác và kiếm thu nhập thụ động cùng chúng tôi.",
    "Thông báo tuyển sinh đi học tại Căm-pu-chia diện Hiệp định năm 2025 · THÔNG BÁO TUYỂN SINH ĐI HỌC TẠI MA-RỐC NĂM 2025 · THÔNG BÁO TUYỂN SINH ĐI HỌC TẠI MÔNG CỔ ...",
    "7 ngày trước — qq188bet! Nền tảng xổ số trực tuyến với tỷ lệ hoa hồng cho đại lý cao nhất. Hợp tác và kiếm thu nhập thụ động cùng chúng tôi. Trở thành đối tác ...",
    "bốn đôi thông chặt được gì? cách dùng bốn đôi thông hiệu quả. 19 thg 10, 2024 - bốn đôi thông là một tổ hợp bài đặc biệt trong các trò chơi bài miền nam, đặc biệt là tiến lên. khi một người chơi sở hữu bốn đôi bài giống nhau ...",
    "Đăng nhập VIC",
    "GGBet",
    "Kết quả CCHC",
    "Sầu riêng Ri6",
    "Vú sữa tím",
    "Trà C2",
    "Cơn_sốt_vàng_bit Cơn_sốt_vàng_bit-Được thi đấu trên sân nhà ở trận chung kết lượt về đội tuyển Thái Lan nhập cuộc với tinh thần thần rất cao. Đội chủ sân Rajamangala cầm",
    "kwin congan.khanhhoa.gov.vn",
    "nohu69 congan.khanhhoa.gov.vn",
    "Cơn_sốt_vàng_bit",
    "+85 sản phẩm sàn gỗ Malaysia siêu chịu nước giá tốt nhất",
    "Sàn gỗ Malaysia chính hãng, chất lượng, giá cả cạnh tranh được phân phối bởi JANHOME là hệ thống bán lẻ sàn gỗ, sàn nhựa, giấy dán tường, vật liệu nội thất ...",
    "+85 sản phẩm sàn gỗ Malaysia siêu chịu nước giá tốt nhất Sàn gỗ Malaysia chính hãng, chất lượng, giá cả cạnh tranh được phân phối bởi JANHOME là hệ thống bán lẻ sàn gỗ, sàn nhựa, giấy dán tường, vật liệu nội thất ...",
    "giọng_nữ_trầm",
    "giọng_nữ_trầm-Chủ tịch HĐQT Bệnh viện thẩm mỹ Sao Hàn chia sẻ rằng, thẩm mỹ hay phẫu thuật thẩm mỹ thì yêu cầu, mong muốn đầu tiên chắc chắn phải đẹp.",
    "giọng_nữ_trầm giọng_nữ_trầm-Chủ tịch HĐQT Bệnh viện thẩm mỹ Sao Hàn chia sẻ rằng, thẩm mỹ hay phẫu thuật thẩm mỹ thì yêu cầu, mong muốn đầu tiên chắc chắn phải đẹp.",
    "Hệ thống QLVBDH: Trang chủ",
    "Đăng nhập Đăng nhập. Chuyển tới trang đầy đủ. Đăng nhập hệ thống. Phím chuyển chữ hoa đang bật. Đăng nhập. Đăng nhập qua hệ thống xác thực TP. Cần Thơ.",
    "Hệ thống QLVBDH: Trang chủ Đăng nhập Đăng nhập. Chuyển tới trang đầy đủ. Đăng nhập hệ thống. Phím chuyển chữ hoa đang bật. Đăng nhập. Đăng nhập qua hệ thống xác thực TP. Cần Thơ.",
    "Đăng nhập VIC",
    "Tên đăng nhập : Mật khẩu : Đăng nhập. Thoát. VIC 6.5 Được phát triển bởi công ty CINOTEC 282 Lê Quang Định, Phường 11, Quận Bình Thạnh, TP HCM."
    "Đăng nhập VIC Tên đăng nhập : Mật khẩu : Đăng nhập. Thoát. VIC 6.5 Được phát triển bởi công ty CINOTEC 282 Lê Quang Định, Phường 11, Quận Bình Thạnh, TP HCM."
    "Đối tượng bảo trợ",
    "Trung tâm Bảo trợ Xã hội là nơi quản lý chăm sóc, nuôi dưỡng điều trị đối tượng Bảo trợ theo quy định của nhà nước - Địa chỉ: Khu vực Bình Hòa A, ...",
    "Đối tượng bảo trợ Trung tâm Bảo trợ Xã hội là nơi quản lý chăm sóc, nuôi dưỡng điều trị đối tượng Bảo trợ theo quy định của nhà nước - Địa chỉ: Khu vực Bình Hòa A, ...",
    "Đăng ký doanh nghiệp qua mạng điện tử",
    "Hình I.1.2. Biểu mẫu nhập thông tin đăng ký tài khoản. Quý doanh nghiệp nhập đầy đủ các trường thông tin của biểu mẫu theo Hình.",
    "Đăng ký doanh nghiệp qua mạng điện tử Hình I.1.2. Biểu mẫu nhập thông tin đăng ký tài khoản. Quý doanh nghiệp nhập đầy đủ các trường thông tin của biểu mẫu theo Hình.",
    "Đăng nhập hệ thống",
    "Đăng nhập hệ thống. Lưu tài khoản đăng nhập. Đăng nhập. Quên mật khẩu.",
"Đăng nhập hệ thống Đăng nhập hệ thống. Lưu tài khoản đăng nhập. Đăng nhập. Quên mật khẩu.",
    "Liên đoàn Taekwondo TP Cần Thơ tăng cường chuyển đổi ...",
    "Mục tiêu của Liên đoàn Taekwondo TP Cần Thơ trong năm 2024 là đẩy mạnh chuyển đổi số, xây dựng mô hình quản lý phù hợp điều kiện, xu thế và quy định của Liên ...",
    "Liên đoàn Taekwondo TP Cần Thơ tăng cường chuyển đổi ... Mục tiêu của Liên đoàn Taekwondo TP Cần Thơ trong năm 2024 là đẩy mạnh chuyển đổi số, xây dựng mô hình quản lý phù hợp điều kiện, xu thế và quy định của Liên ...",
    "MẪU CHUYỆN “ĐÔI DÉP BÁC HỒ”",
    "3 thg 10, 2023 — Một anh nhanh tay giành lấy chiếc dép, giơ lên nhưng ngớ ra, lúng túng. Anh bên cạnh liếc thấy, “vượt vây” chạy biến… Bác phải giục:“Ơ kìa, ngắm ...",
    "MẪU CHUYỆN “ĐÔI DÉP BÁC HỒ” 3 thg 10, 2023 — Một anh nhanh tay giành lấy chiếc dép, giơ lên nhưng ngớ ra, lúng túng. Anh bên cạnh liếc thấy, “vượt vây” chạy biến… Bác phải giục:“Ơ kìa, ngắm ...",
    "Ảnh hưởng của các loại thức ăn đến sinh trưởng và tỉ lệ ...",
    "6 thg 12, 2022 — là một trong những loài chân bụng nước ngọt được tìm thấy trong ao nước ngọt, vũng, bể, hồ, đầm lầy, ruộng lúa và đôi khi ở sông suối. Hiện nay, ...",
    "Ảnh hưởng của các loại thức ăn đến sinh trưởng và tỉ lệ ... 6 thg 12, 2022 — là một trong những loài chân bụng nước ngọt được tìm thấy trong ao nước ngọt, vũng, bể, hồ, đầm lầy, ruộng lúa và đôi khi ở sông suối. Hiện nay, ...",
    "Lịch sử hình thành",
    "Ngày 01 tháng 01 năm 2004, tỉnh Cần Thơ được chia tách thành 02 đơn vị hành chính là TP. Cần Thơ và tỉnh Hậu Giang. Bảo tàng tỉnh Cần đổi tên cho phù hợp với ...",
    "Lịch sử hình thành Ngày 01 tháng 01 năm 2004, tỉnh Cần Thơ được chia tách thành 02 đơn vị hành chính là TP. Cần Thơ và tỉnh Hậu Giang. Bảo tàng tỉnh Cần đổi tên cho phù hợp với ...",
    "Hội Cựu chiến binh thành phố Cần Thơ",
    "Thông tin liên hệ. Hội Cựu chiến binh - Thành phố Cần Thơ Địa chỉ : 22 Trần Văn Hoài, P.Xuân Khánh, Q.Ninh Kiều, TP Cần Thơ Điện thoại: (0710) 3832735",
    "Hội Cựu chiến binh thành phố Cần Thơ Thông tin liên hệ. Hội Cựu chiến binh - Thành phố Cần Thơ Địa chỉ : 22 Trần Văn Hoài, P.Xuân Khánh, Q.Ninh Kiều, TP Cần Thơ Điện thoại: (0710) 3832735",
    "văn hóa Địa điểm Chiến thắng Ông Đưa năm 1960",
"DI TÍCH LỊCH SỬ - VĂN HÓA ĐỊA ĐIỂM CHIẾN THẮNG ÔNG ĐƯA NĂM 1960 ... Di tích lịch sử - văn hóa Địa điểm Chiến thắng Ông Đưa năm 1960 tọa lạc tại ấp Định Khánh A, ...",
    "văn hóa Địa điểm Chiến thắng Ông Đưa năm 1960 DI TÍCH LỊCH SỬ - VĂN HÓA ĐỊA ĐIỂM CHIẾN THẮNG ÔNG ĐƯA NĂM 1960 ... Di tích lịch sử - văn hóa Địa điểm Chiến thắng Ông Đưa năm 1960 tọa lạc tại ấp Định Khánh A, ...",
    "Ước ao của thiếu nhi qua bài hát",
    "ƯỚC AO CỦA THIẾU NHI QUA BÀI HÁT “EM MƠ GẶP BÁC HỒ” CỦA NHẠC SĨ XUÂN GIAO. Nhạc sĩ Xuân Giao quê gốc ở Như Quỳnh, Văn Lâm, Hưng Yên, sinh năm 1932 tại Hải ...",
    "Ước ao của thiếu nhi qua bài hát ƯỚC AO CỦA THIẾU NHI QUA BÀI HÁT “EM MƠ GẶP BÁC HỒ” CỦA NHẠC SĨ XUÂN GIAO. Nhạc sĩ Xuân Giao quê gốc ở Như Quỳnh, Văn Lâm, Hưng Yên, sinh năm 1932 tại Hải ...",
    "BẢNG THANH TOÁN PHỤ CẤP CÁN BỘ CÔNG ĐOÀN",
    "BẢNG THANH TOÁN PHỤ CẤP CÁN BỘ CÔNG ĐOÀN · Các biểu mẫu tài chính Công đoàn cơ sở · Mẫu hướng dẫn Công đoàn cơ sở · Mẫu biểu dự toán, quyết toán tài chính CĐ ...",
    "BẢNG THANH TOÁN PHỤ CẤP CÁN BỘ CÔNG ĐOÀN BẢNG THANH TOÁN PHỤ CẤP CÁN BỘ CÔNG ĐOÀN · Các biểu mẫu tài chính Công đoàn cơ sở · Mẫu hướng dẫn Công đoàn cơ sở · Mẫu biểu dự toán, quyết toán tài chính CĐ ...",
    "dung tin co ay tap 2 Sòng bạc thông thường của Việt Nam",
    "dung tin co ay tap 2 -Xuáº¥t hiá»‡n cÃ¹ng kiá»ƒu tÃ³c layer vuá»'t ngÆ°á»£c vá»›i pháº§n tÃ³c tá»« hai mang tai Ä'á» u Ä'Æ°á»£c háº¥t ngÆ°á»£c ra sau vÃ ...",
    "dung tin co ay tap 2 Sòng bạc thông thường của Việt Nam dung tin co ay tap 2 -Xuáº¥t hiá»‡n cÃ¹ng kiá»ƒu tÃ³c layer vuá»'t ngÆ°á»£c vá»›i pháº§n tÃ³c tá»« hai mang tai Ä'á» u Ä'Æ°á»£c háº¥t ngÆ°á»£c ra sau vÃ ...",
    "Thông tin truy nã, đình nã",
    "Thông tin truy nã, đình nã · Thông báo · Thông tin truy nã · Liên kết webiste · Thăm dò ý kiến · Số lượt truy cập. Trong ngày: Tất cả:.","Cờ bạc"
    "Thông tin truy nã, đình nã Thông tin truy nã, đình nã · Thông báo · Thông tin truy nã · Liên kết webiste · Thăm dò ý kiến · Số lượt truy cập. Trong ngày: Tất cả:.","Cờ bạc"
    "+79 sản phẩm sàn gỗ Florton chính hãng, chất lượng, giá rẻ",
    "SÀN GỖ FLORTON chất lượng, giá rẻ, đạt tiêu chuẩn Châu Âu được cung cấp bởi JANHOME là hệ thống bán hàng tại kho cung cấp vật liệu sàn gỗ giấy dán tường ...",
"+79 sản phẩm sàn gỗ Florton chính hãng, chất lượng, giá rẻ SÀN GỖ FLORTON chất lượng, giá rẻ, đạt tiêu chuẩn Châu Âu được cung cấp bởi JANHOME là hệ thống bán hàng tại kho cung cấp vật liệu sàn gỗ giấy dán tường ...",
    "Trang chủ - Cần Thơ",
    "Bộ Công Thương vừa ban hành Thông tư quy định việc nhập khẩu mặt hàng gạo và lá thuốc lá khô có xuất xứ từ Campuchia theo hạn ngạch thuế quan năm 2023 và 2024.",
    "Trang chủ - Cần Thơ Bộ Công Thương vừa ban hành Thông tư quy định việc nhập khẩu mặt hàng gạo và lá thuốc lá khô có xuất xứ từ Campuchia theo hạn ngạch thuế quan năm 2023 và 2024.",
    "Login",
    "???login.label.loginheading.left??? ???login.label.userid??? ???login.label.password??? Help. Product documentation · Product wiki · Media gallery ...",
    "Login ???login.label.loginheading.left??? ???login.label.userid??? ???login.label.password??? Help. Product documentation · Product wiki · Media gallery ...",
    "Đăng ký tư vấn từ QR code",
    "Đăng ký tư vấn từ QR code. Xoá Dán (Paste)",
    "Đăng ký tư vấn từ QR code Đăng ký tư vấn từ QR code. Xoá Dán (Paste)",
    "GGBET",
    "ggbet",
    "+79 sản phẩm sàn gỗ Florton chính hãng, chất lượng, giá rẻ",
    "SÀN GỖ FLORTON chất lượng, giá rẻ, đạt tiêu chuẩn Châu Âu được cung cấp bởi JANHOME là hệ thống bán hàng tại kho cung cấp vật liệu sàn gỗ giấy dán tường ...",
    "+79 sản phẩm sàn gỗ Florton chính hãng, chất lượng, giá rẻ SÀN GỖ FLORTON chất lượng, giá rẻ, đạt tiêu chuẩn Châu Âu được cung cấp bởi JANHOME là hệ thống bán hàng tại kho cung cấp vật liệu sàn gỗ giấy dán tường ...",
    "kênh-xoilac",
    "kênh-xoilac-Ngày 3.1, HĐND H.Quế Sơn khóa XII (nhiệm kỳ 2021 - 2026) tổ chức kỳ họp chuyên đề thứ 15 để bầu các chức danh lãnh đạo chủ chốt sau sáp nhập ...",
    "kênh-xoilac kênh-xoilac-Ngày 3.1, HĐND H.Quế Sơn khóa XII (nhiệm kỳ 2021 - 2026) tổ chức kỳ họp chuyên đề thứ 15 để bầu các chức danh lãnh đạo chủ chốt sau sáp nhập ...",
    "cách lai tạo màu lông gà 0209",
    "cách lai tạo màu lông gà 0209-Kế tiếp là cung điện Minos tọa lạc ở Crete, (Hy Lạp), được cho là xây dựng vào khoảng năm 1700 trước Công nguyên, ...",
    "cách lai tạo màu lông gà 0209 cách lai tạo màu lông gà 0209-Kế tiếp là cung điện Minos tọa lạc ở Crete, (Hy Lạp), được cho là xây dựng vào khoảng năm 1700 trước Công nguyên, ..."
]

for i, text in enumerate(example_texts):
    predicted_label = predict_text_class(text)

    print(f"  Input: '{text}'")
    print(f"  Predicted: {predicted_label}")
    print()

# results = []

# for i, text in enumerate(example_texts):
#     predicted_label = predict_text_class(text)
#     results.append(f"Input: '{text}'\nPredicted: {predicted_label}\n")

# # Save to file
# with open("predictions.txt", "w", encoding="utf-8") as f:
#     f.write("\n".join(results))

# print("Predictions saved to predictions.txt")